In [1]:
import sympy as sp

(x, r, H, m, pt, p_raise, t, S, ins, C,
 P_cap) = sp.symbols('x r H m prop_t prop_raise fed_t S ins C P_cap')
N, T = sp.symbols('N T', integer=True)

* {{x}} : Downpayment, what we want to optimize.
* {{r}} : Mortgage monthly rate.
* {{H}} : House price
* {{m}} : Monthly mortgage payment.
* {{pt}} : Yearly property tax.
* {{p_raise}} : Yearly property value increase, by county.
* {{t}} : Marginal yearly tax rate.
* {{S}} : Standard deduction (taxes). 
* {{ins}}: Monthly insurance.
* {{T}} : Mortgage term in years.
* {{N}} : Sale date in years.
* {{C}} : Monthly capital available
* {{P_cap}} : Cap on principal interest deduction.

In [2]:
def sp_max(var1, var2):
    return sp.Piecewise((var1, var1 >= var2), (var2, var1 < var2))


def sp_min(var1, var2):
    return sp.Piecewise((var1, var1 <= var2), (var2, var1 > var2))


k = sp.Symbol('k', integer=True)
P_k = sp.Symbol('P_k')
P_k_sub = (1 + r)**k * (H - x) - (m / r) * ((1 + r)**k - 1)

I_k = sp.Symbol('I_k')
I_k_sub = r * sp_min(P_k, P_cap)

year_num = sp.Symbol('Y', integer=True)
yearly_property_tax = pt * (1 + p_raise)**year_num * H
deductible_year = yearly_property_tax + sp.Sum(I_k, (k, 12 * year_num, 12 *
                                                     (year_num + 1) - 1))

deduction_year = sp_max(deductible_year - S, 0) * t

year_k = sp.floor(k / 12)

deduction_k = sp.Rational(1, 12) * deduction_year.subs(year_num, year_k)
property_tax_k = sp.Rational(1, 12) * yearly_property_tax.subs(
    year_num, year_k)

In [3]:
# Creating equations for display:
(disp_deductible_year, disp_deduction_year,
 disp_deduction_k) = sp.symbols('deductible_Y deduction_Y deduction_k')

disp_eq1 = sp.Eq(P_k, P_k_sub)
disp_eq2 = sp.Eq(I_k, I_k_sub)
disp_eq3 = sp.Eq(disp_deductible_year, deductible_year)
disp_eq4 = sp.Eq(disp_deduction_year, deduction_year)
disp_eq5 = sp.Eq(disp_deduction_k, deduction_k)

Remaining principal for the {{k}}<sup>th</sup> month:

{{disp_eq1}}

Interest for the {{k}}<sup>th</sup> month:

{{disp_eq2}}

Deductible portion for year {{year_num}}:

{{disp_eq3}}

Deductions for year {{year_num}}:

{{disp_eq4}}

Deductions for the {{k}}<sup>th</sup> month:

{{disp_eq5}}

In [4]:
substitution_dict = {
    N: 7,
    T: 30,
    r: (3.125 / 100) / 12,
    pt: 1.3 / 100,
    p_raise: 1.5 / 100,
    t: 32 / 100,
    H: 1.4e6,
    C: 5900,
    S: 24400,
    ins: 100,
    P_cap: 750000
}

In [5]:
k_subs = {I_k: I_k_sub, P_k: P_k_sub}

deduction_N = deduction_k.subs(k_subs).subs(k, 12 * N)
property_tax_N = property_tax_k.subs(k_subs).subs(k, 12 * N)
eq1 = sp.Eq(m + property_tax_N - deduction_N + ins, C).doit()

P_T = P_k_sub.subs(k, 12 * T)
eq2 = sp.Eq(P_T, 0).doit()

In [6]:
eq1 = eq1.subs(substitution_dict)
eq2 = eq2.subs(substitution_dict)
vals = sp.nsolve([eq1, eq2], [m, x], [1, 1])

In [7]:
m_star = vals[0]
x_star = vals[1]

all_substitutions = {**substitution_dict, **{m: m_star, x: x_star}}
monthly_tax = property_tax_N.subs(all_substitutions)
monthy_deductions = deduction_N.subs(all_substitutions)

Monthly mortgage payment ({{m}}): {{'{:,.0f}'.format(m_star)}}

Downpayment ({{x}}): {{'{:,.0f}'.format(x_star)}}

Monthly tax : {{'{:,.0f}'.format(monthly_tax)}}

Monthly deduction ({{disp_deduction_k}}): {{'{:,.0f}'.format(float(monthy_deductions))}}

## Total gains in the buyers case

In [8]:
h, s, espp, closing_costs = sp.symbols('h s espp closing-costs')

appreciation_substitutions = {
    **all_substitutions,
    **{
        h: 2 / 100,
        s: 8 / 100,
        espp: 1900 * 6,
        closing_costs: 30_000
    }
}
appreciated_house_price = (1 + h)**N * H
gains = appreciated_house_price - P_k_sub.subs(k, 12 * N) - closing_costs

House price after {{appreciation_substitutions[N]}} years: {{'{:,.0f}'.format(appreciated_house_price.subs(appreciation_substitutions).doit())}}

Total gains with house appreciation of {{appreciation_substitutions[h]*100}}%: {{'{:,.0f}'.format(gains.subs(appreciation_substitutions).doit())}}

#### Total gains in the renters case

In [9]:
appreciated_downpayment = (1 + s)**N * appreciation_substitutions[x]

Downpayment after {{appreciation_substitutions[N]}} years:
{{'{:,.0f}'.format(appreciated_downpayment.subs(appreciation_substitutions).doit())}}

In [10]:
s_12 = sp.sqrt(1 + s) - 1

espp_gains = sp.Sum((1 + s_12)**k * espp, (k, 0, 2 * N))

ESPP after {{appreciation_substitutions[N]}} years: {{'{:,.0f}'.format(espp_gains.subs(appreciation_substitutions).doit())}}

Total gains with stock appreciation of {{appreciation_substitutions[s]*100}}%: 
{{'{:,.0f}'.format((appreciated_downpayment + espp_gains).subs(appreciation_substitutions).doit())}}

## More Downpayment on mortgage

In [11]:
appreciation_substitutions[x] = 2 * appreciation_substitutions[x]

Downpayment:
{{'{:,.0f}'.format(appreciation_substitutions[x])}}

House price after {{appreciation_substitutions[N]}} years: {{'{:,.0f}'.format(appreciated_house_price.subs(appreciation_substitutions).doit())}}

Total gains with house appreciation of {{appreciation_substitutions[h]*100}}%: {{'{:,.0f}'.format(gains.subs(appreciation_substitutions).doit())}}

### Compared to investing that money

In [12]:
appreciation_substitutions[x] = x_star

Downpayment:
{{'{:,.0f}'.format(appreciation_substitutions[x])}}

House price after {{appreciation_substitutions[N]}} years: {{'{:,.0f}'.format(appreciated_house_price.subs(appreciation_substitutions).doit())}}

Total gains with house appreciation of {{appreciation_substitutions[h]*100}}%: {{'{:,.0f}'.format(gains.subs(appreciation_substitutions).doit())}}

Total gains with stock appreciation of {{appreciation_substitutions[s]*100}}%: 
{{'{:,.0f}'.format((appreciated_downpayment + espp_gains).subs(appreciation_substitutions).doit())}}

Sum:
{{'{:,.0f}'.format((gains + appreciated_downpayment + espp_gains).subs(appreciation_substitutions).doit())}}